In [2]:
import sys, glob, os
SPARK_HOME=os.environ['SPARK_HOME']
sys.path.append(SPARK_HOME + "/python")
sys.path.append(glob.glob(SPARK_HOME + "/python/lib/py4j*.zip")[0])
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext
sql = spark.sql
print(sc.uiWebUrl, spark.version)

http://192.168.1.4:4040 2.4.4


In [3]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window, WindowSpec

In [4]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"),
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]


#df = spark.createDataFrame(data)
#df.columns = ["Product","Amount","Country"]
#df.show()

In [5]:
import pandas as pd

In [6]:
df_pandas = pd.DataFrame.from_records(data, columns = ["Product","Amount","Country"])
df_pandas.head()

,Product,Amount,Country
0,Banana,1000,USA
1,Carrots,1500,USA
2,Beans,1600,USA
3,Orange,2000,USA
4,Orange,2000,USA


In [7]:
df_spark = spark.createDataFrame(df_pandas)
df_spark.show()

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+



In [8]:
pivotDF = df_spark.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.show()

+-------+------+-----+------+----+
|Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  null| 4000|  null|4000|
|  Beans|  null| 1500|  2000|1600|
| Banana|  2000|  400|  null|1000|
|Carrots|  2000| 1200|  null|1500|
+-------+------+-----+------+----+



In [9]:
countries = ["USA","China","Canada","Mexico"]
pivotDF = df_spark.groupBy("Product").pivot("Country", countries).sum("Amount")
pivotDF.show()

+-------+----+-----+------+------+
|Product| USA|China|Canada|Mexico|
+-------+----+-----+------+------+
| Orange|4000| 4000|  null|  null|
|  Beans|1600| 1500|  null|  2000|
| Banana|1000|  400|  2000|  null|
|Carrots|1500| 1200|  2000|  null|
+-------+----+-----+------+------+



In [10]:
unPivotDF = (pivotDF.select(
    col("Product"),
    expr("stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"))
.where("Total is not null"))

unPivotDF.show()

+-------+-------+-----+
|Product|Country|Total|
+-------+-------+-----+
| Orange|  China| 4000|
|  Beans|  China| 1500|
|  Beans| Mexico| 2000|
| Banana| Canada| 2000|
| Banana|  China|  400|
|Carrots| Canada| 2000|
|Carrots|  China| 1200|
+-------+-------+-----+



In [24]:
spark.sql("select distinct product from fruit").show()

+-------+
|product|
+-------+
| Orange|
|  Beans|
| Banana|
|Carrots|
+-------+



In [11]:
df_spark.registerTempTable("fruit")

In [23]:
sql("""SELECT * FROM (select PRODUCT, COUNTRY,Amount from fruit )
PIVOT(avg(Amount) for product in ('Orange', 'Beans', 'Carrots'))""").show()

+-------+------+------+-------+
|COUNTRY|Orange| Beans|Carrots|
+-------+------+------+-------+
|  China|4000.0|1500.0| 1200.0|
|    USA|2000.0|1600.0| 1500.0|
| Mexico|  null|2000.0|   null|
| Canada|  null|  null| 2000.0|
+-------+------+------+-------+



In [26]:
sql("""SELECT * FROM (select PRODUCT, COUNTRY,Amount from fruit )
PIVOT(avg(Amount) for product in (select distinct product from fruit))""").show()

ParseException: "\nmismatched input 'FROM' expecting <EOF>(line 1, pos 9)\n\n== SQL ==\nSELECT * FROM (select PRODUCT, COUNTRY,Amount from fruit )\n---------^^^\nPIVOT(avg(Amount) for product in (select distinct product from fruit))\n"